Nets are defined as having one source and multiple target.  
Note that $x_{i}x_{j} - 2x_{i}y_{ij} - 2x_{j}y_{ij} + 3y_{ij}$ penalty enforces $y_{ij} = x_{i}x_{j}$ (the equality makes it vanish). See "Quadratic reformulations of nonlinear
binary optimization problems" by Martin Anthony et. al.

- Constraint to set one node to 1 for the ending node: $(1-\sum_i y_i + \sum_{i\neq j} 2y_iy_j)$

- Constraint to set other nodes to 0 or 2: $(\sum_jy_j)(2-\sum_jy_j)^2= \sum_i y_i-2\sum_{i\neq j}y_iy_j+6\sum_{i\neq j,i\neq k,j\neq j}y_iy_jy_k$. Adding ancilla variables to make it QUBO:
$\sum_i y_i-2\sum_{i\neq j}y_iy_j+6\sum_{i\neq j,i\neq k,j\neq k}[w_{i,j}y_k]+ \sum_{i\neq j,i\neq k,j\neq k}[y_iy_j-2w_{i,j}(y_i+y_j)+3w_{i,j}]$

- Constraint to set one node to $n$ for the starting node: $(\sum_iy_i-n)^2=n^2-(2n-1)\sum_i y_i+\sum_{i\neq j} 2y_iy_j$

- Minimum distance objective function:  $w\sum_{i} y_i$ if $w$ is not there this term dwarves the terms enforcing the constraints. While the constraints only add a penalty or reward of 1/-1, the objective function's penalty grows linearly with the routs length. Therefore, it is imperetive that we nulify the effect of this growth with a small weight. A weight too big wont do shit and a weight too small will make the adiabatic gap very tiny and inceases the error rate. 

In [ ]:
import sys
from pathlib import Path
pp = str(Path('.').absolute().parent)
if pp not in sys.path:
    sys.path.append(pp)

In [ ]:
#import the necessary shit
import matplotlib.pyplot as plt
import numpy as np
import skopt
import skopt.space as space
import requests

import dwave_networkx as dnx
import networkx as nx
import minorminer
import neal
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import FixedEmbeddingComposite
from dwave.cloud import exceptions as dwave_exceptions 
import dimod

from qpr.routing import (
    NetList, Route, get_qubo, edge_list_from_qubo_answer_dict, qubo_violation
)
from qpr.quantum_utils import find_embedding_minorminer, get_all_min_energy
from qpr.notebook_utils import make_ax_grid

%matplotlib inline

In [ ]:
edges = [
    (6, 7), (7, 8), (6, 3), (7, 4), (8, 5),
    (3, 4), (4, 5), (3, 0), (4, 1), (5, 2),
    (0, 1), (1, 2),
]
rev_edges = [(j, i) for i, j in edges]
edges = edges + rev_edges
edges = edges + [
    ('o0', 0),  # (0, 'i0'), (1, 'i1'), ('o1', 1),
    # (2, 'i2'), ('o2', 2), (3, 'i3'), ('o3', 3),
    # (5, 'i5'), ('o5', 5), (6, 'i6'), ('o6', 6),
    (7, 'i7'),  # ('o7', 7), (8, 'i8'), ('o8', 8),
]

# cartesian frame, i.e. (x, y) pairs, with origin at bottom left
pos = {
    0: (0, 0), 1: (1, 0), 2: (2, 0), 3: (0, 1),
    4: (1, 1), 5: (2, 1), 6: (0, 2), 7: (1, 2),
    8: (2, 2), 'o0': (-.8, -.4),  # 'i0': (-.4, -.8),
    # 'i1': (.8, -1), 'o1': (1.2, -1), 
    # 'i2': (2.6, -.4), 'o2': (2.4, -.6),
    # 'i3': (-1, .8), 'o3': (-1, 1.2),
    # 'i5': (3, .8), 'o5': (3, 1.2),
    # 'i6': (-.4, 2.6), 'o6': (-.6, 2.4),
    'i7': (.8, 3),  # 'o7': (1.2, 3),
    # 'i8': (2.4, 2.6), 'o8': (2.6, 2.4),
}
super_graph = nx.DiGraph(edges)
nx.draw_networkx(super_graph, pos=pos)

In [ ]:
route_obj = Route(
    super_graph,
    NetList([('s1', ['t11']), ]),
    set(),
    dict(s1='o0', t11='i7'),
    pos
)
route_obj.draw_arch()

In [ ]:
# H = G.to_undirected(as_view=True) is the fastest but its H.number_of_edges() or len(H.edges)
# returns the number of edges in G (oddly enough, iterating through the edges 
# i.e. for i in or H.edges or list(H) yields the correct set of edges:|).
# nx.Graph(G) does a shallow copy and G.to_undirected() does a deep copy.

# Doing this with the directed graph doubles the number of variables..
Q = get_qubo(nx.Graph(route_obj.arch_graph), route_obj.placed_netlist)

In [ ]:
%%time

dwave_sampler = DWaveSampler(solver={'lower_noise': True, 'qpu': True})
A = dwave_sampler.edgelist
embedding, chain_len = find_embedding_minorminer(Q, A, num_tries=5)
# the shortest chain_len I've seen with num_tries=1000 is 5
# (SP: takes 2.5 mins on my machine)
display(chain_len)

# QPU

In [ ]:
fixed_sampler = FixedEmbeddingComposite(
    DWaveSampler(solver={'lower_noise': True, 'qpu': True}), embedding
)

# Q = get_qubo(
#     nx.Graph(route_obj.arch_graph), route_obj.placed_netlist, w_obj=0.1, w_target=100,
#     w_nonterm=1, w_source=100, w_and=6,
# )
Q = get_qubo(
    nx.Graph(route_obj.arch_graph), route_obj.placed_netlist, w_obj=1, w_target=2,
    w_nonterm=1, w_source=3, w_and=5,
)
q_response = fixed_sampler.sample_qubo(
    Q, chain_strength=4, annealing_time=500, auto_scale=True, num_reads=1000
)

display(q_response.first)
best_q_answer = q_response.first.sample

score_vec = qubo_violation(
    best_q_answer, route_obj.all_source_nodes, route_obj.all_target_nodes,
    route_obj.all_nt_nodes
)
display(f'violations: {score_vec}')

best_q_answer = [eval(i[1:]) for i in best_q_answer if best_q_answer[i] == 1 and i[0] == 'y']
display(best_q_answer)

In [ ]:
fig, ax = plt.subplots()
route_obj.edge_list = best_q_answer
route_obj.draw(draw_netlist=False, draw_subgraph=False, axes=[ax, None, None])
ax.set_title(f'{route_obj.score()}');

# Exact solver

In [ ]:
%%time

# TODO this eats up the RAM if Q is large
# run at your own risk, open the system monitor or something
# and keep an eye on the memory usage..
assert len(Q) < 100

exact_response = dimod.ExactSolver().sample_qubo(Q)
display(exact_response.record)

In [ ]:
min_energy_sols, _ = get_all_min_energy(exact_response)
fig, axes = make_ax_grid(len(min_energy_sols))
display(len(min_energy_sols))

for ax, answer_dict in zip(axes.flat, min_energy_sols):
    display(answer_dict)
    route_obj.edge_list = edge_list_from_qubo_answer_dict(answer_dict)
    display(route_obj.edge_list)
    route_obj.draw(draw_netlist=False, draw_subgraph=False, axes=[ax, None, None])
    display(route_obj.edge_list, route_obj.arch_graph.edges)
    #nx.draw_networkx_edges(route_obj.arch_graph, edgelist=route_obj.edge_list, ax=ax, pos=route_obj.pos, edge_color='r', node_size=900, width=8)


fig.tight_layout()

In [ ]:
verify_and(answer_dict), min_energy_sols, Q

# Simulated Annealing

In [ ]:
import time

def optimize_qannealer(args):
    w_obj, w_target, w_nonterm, w_source, w_and = args
    Q = get_qubo(
        nx.Graph(route_obj.arch_graph), route_obj.placed_netlist, w_obj=w_obj,
        w_target=w_target, w_nonterm=w_nonterm, w_source=w_source, w_and=w_and,
    )
    
    #embedding, _ = find_embedding_minorminer(Q, A, num_tries=5)

    count = 0
    succeeded = False
    while not succeeded:
        try:
            count += 1
            response = fixed_sampler.sample_qubo(
                Q, chain_strength=4, annealing_time=500, auto_scale=True, num_reads=1000
            )
            succeeded = True
        except dwave_exceptions.RequestTimeout as e:
            display(f'--------------------failed count-------------------------: {count}')
            succeeded = False
            time.sleep(5)
        except requests.exceptions.ConnectionError as e:
            display(f'--------------------failed count-------------------------: {count}')
            succeeded = False
            time.sleep(5)            
        except Exception as e:
            display(f'type: {type(e)}')
            display(e)
            raise(e)
    
    result = response.first.sample
    score_dict = qubo_violation(
        result, route_obj.all_source_nodes, route_obj.all_target_nodes,
        route_obj.all_nt_nodes
    )
    score = score_dict['obj'] + 3 * np.sum([score_dict[i] for i in ('s', 't', 'and', 'nt')])
    
    # result = [eval(i[1:]) for i in result if result[i] == 1 and i[0] == 'y']
    
    # route_obj.edge_list = result
    # score = route_obj.score()
    # if score == np.inf:
    #     score = 100000000
    
    return score

In [ ]:
%%time

### stick to uniform priors and linear exp to avoid known skopt bugs..sigh
search_space = [
    space.Real(1e-1, 10, prior='uniform', name='w_obj'),
    space.Real(1e-1, 10, prior='uniform', name='w_target'),
    space.Real(1e-1, 10, prior='uniform', name='w_nonterm'),
    space.Real(1e-1, 10, prior='uniform', name='w_source'),
    space.Real(1e-1, 10, prior='uniform', name='w_and'),
]

res_gp = skopt.gp_minimize(
    optimize_qannealer, search_space, acq_func='gp_hedge', n_calls=200,
    n_random_starts=150, random_state=2, verbose=True,
)
res_gp.x, res_gp.fun

In [ ]:
gp_best_func_val = np.array(
    [np.min(res_gp.func_vals[:i]) for i in range(1, res_gp.func_vals.size+1)]
)
plt.plot(gp_best_func_val, label='GP best')
plt.scatter(np.arange(res_gp.func_vals.size), res_gp.func_vals, label='GP eval')
plt.xlabel('Num eval')
plt.ylabel('score')
plt.legend();

Q = get_qubo(
    nx.Graph(route_obj.arch_graph), route_obj.placed_netlist, w_obj=res_gp.x[0],
    w_target=res_gp.x[1], w_nonterm=res_gp.x[2], w_source=res_gp.x[3], w_and=res_gp.x[4],
)
response = fixed_sampler.sample_qubo(
    Q, chain_strength=4, annealing_time=500, auto_scale=True, num_reads=1000
)
response = response.first.sample
route_obj.edge_list = edge_list_from_qubo_answer_dict(response)

fig, ax = plt.subplots()
route_obj.draw(draw_netlist=False, draw_subgraph=False, axes=[ax, None, None])
ax.set_title(f'{route_obj.score()}');

In [ ]:
res_gp.x

# hybrid solution

In [ ]:
import hybrid

# Construct a problem
offset=0.0
#vartype = dimod.BINARY
bqm = dimod.BinaryQuadraticModel.from_qubo(Q, offset)

# Define the workflow
iteration = hybrid.RacingBranches(
    hybrid.InterruptableTabuSampler(),
    hybrid.EnergyImpactDecomposer(size=2)
    | hybrid.QPUSubproblemAutoEmbeddingSampler()
    | hybrid.SplatComposer()
) | hybrid.ArgMin()
workflow = hybrid.LoopUntilNoImprovement(iteration, convergence=3)

# Solve the problem
init_state = hybrid.State.from_problem(bqm)
final_state = workflow.run(init_state).result()

# Print results
print("Solution: sample={.samples.first}".format(final_state))

In [ ]:
G.draw(edge_labs=False)  # edge_labs=False)

edge_set = G.qubo_answer2node_pairs(answer_dict)
G.highlight_edge_list(edge_set)
